# Week 9: Sentence Level Classification with BERT

Your goal this week is to train a classifier that can predict the CEFR level of any given sentence. In this notebook we will guide you through the process of using 🤗[Hugging Face](https://huggingface.co/) and its transformers library as the training framework, with [Pytorch](https://pytorch.org/) as the deep learning backend, but feel free to use [TensorFlow](https://www.tensorflow.org) if that's what you are more familiar with.

For this assignment we will provide a dataset containing sentences with the corresponding CEFR level, and you have to use BERT and train a sentence classifier with this dataset.

## Prepare your environment

As always, we highly recommend that you install all packages with a virtual environment manager, like [venv](https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/) or [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html), to prevent version conflicts of different packages.  

### Install CUDA
Deep learning is a computionally extensive process. It takes lots of time if relying only on the CPU, especially when it's trained on a large dataset. That's why using GPU instead is generally recommended.  
To use GPU for computation, you have to install [CUDA toolkit](https://developer.nvidia.com/cuda-toolkit) as well as the [cuDNN library](https://developer.nvidia.com/cudnn) provided by NVIDIA.  

If you already had CUDA installed on your machine, then great! You're done here.  
If you don't, you can refer to [Appendix](#Appendix-1-Install-CUDA) to see how to do so.


### Install python packages
The following python packages will be used in this tutorial:

1. `numpy`: for matrix operation
2. `scikit-learn`: for label encoding
3. `datasets`: for data preparation
4. `transformers`: for model loading and finetuing
5. `pytorch`: the backend DL framework
  - Note that the pt version must support the CUDA version you've installed if you want to use GPU.

### Select GPU(s) for your backend

Skip this section if you have no intension of using GPU with tensorflow/pytorch.

In [94]:
import os

# select your GPU. Note that this should be set before you load tensorflow or pytorch.
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# To use multiple GPUs, combine all GPU ID with commas
# e.g. >>> os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,3'

In [95]:
import torch
# Check if any GPU is used
device = torch.device("mps")
torch.cuda.is_available()

False

In [4]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


## Prepare the dataset

Before starting the training, we need to load and process our dataset - but wait, let's decide which model we want to use first.  

In the highly unlikely chance you've never heard of it, [BERT](https://arxiv.org/abs/1810.04805) (**B**idirectional **E**ncoder **R**epresentations from **T**ransformers) is a language model proposed by Google AI in 2018, and it's currently one of the most popular models used in NLP.  
You can learn more about it here:
- [BERT Explained: A Complete Guide with Theory and Tutorial](https://towardsml.com/2019/09/17/bert-explained-a-complete-guide-with-theory-and-tutorial/) by Samia, 2019.


However, we will not directly use BERT in this tutorial, because it's large and takes too long to train. Instead, we'll be using [DistilBert](https://medium.com/huggingface/distilbert-8cf3380435b5), a version of BERT that while light-weight, reserves 95% of its original accuracy.




In [3]:
# the model you want to use. Available models can be found here: https://huggingface.co/models
MODEL_NAME = 'distilbert-base-uncased'

### Load data

Similar to the `transformers` library, `datasets` is also a package by huggingface. It contains many public datasets online and can help us with the data processing.  
We can use `load_dataset` function to read the input `.csv` file provided for this assignment.

Reference:
 - [Official datasets document](https://huggingface.co/docs/datasets)
 - [datasets.load_dataset](https://huggingface.co/docs/datasets/loading.html)

In [59]:
# [ TODO ] load the data using the load_dataset function
from datasets import load_dataset
dataset = load_dataset("data")

Using custom data configuration data-34cb9195d57c06b7
Found cached dataset csv (/Users/rakson/.cache/huggingface/datasets/csv/data-34cb9195d57c06b7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 2/2 [00:00<00:00, 1074.64it/s]


In [60]:
dataset.map(batched=False, with_indices=False)

Loading cached processed dataset at /Users/rakson/.cache/huggingface/datasets/csv/data-34cb9195d57c06b7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7f17c54ea0c07a1c.arrow
Loading cached processed dataset at /Users/rakson/.cache/huggingface/datasets/csv/data-34cb9195d57c06b7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-27af2a2af34d1b00.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'level'],
        num_rows: 20720
    })
    test: Dataset({
        features: ['text', 'level'],
        num_rows: 2300
    })
})

In [21]:
# print(dataset['train'])
# print(dataset['train'][1])
# print(dataset['train']['text'][:5])

Dataset({
    features: ['text', 'level'],
    num_rows: 20720
})
{'text': 'You can contact me by e-mail.', 'level': 'A1'}
['My mother is having her car repaired.', 'You can contact me by e-mail.', 'He had a break for the weekend, and he called me: "I am in London, so, if you want to see me, it\'s the time!"', "Research shows that 40 percent of the program's viewers are aged over 55.", "I'd guess she's about my age."]


In [8]:
print(dataset['train'])
print(dataset['train'][1])
print(dataset['train']['text'][:5])

Dataset({
    features: ['text', 'level'],
    num_rows: 20720
})
{'text': 'You can contact me by e-mail.', 'level': 'A1'}
['My mother is having her car repaired.', 'You can contact me by e-mail.', 'He had a break for the weekend, and he called me: "I am in London, so, if you want to see me, it\'s the time!"', "Research shows that 40 percent of the program's viewers are aged over 55.", "I'd guess she's about my age."]


### Preprocessing

As always, texts should be tokenized, embedded, and padded before being put into the model.  
But not to worry, there are libraries from huggingface to help with this, too.

#### Sentence processing

Different pre-trained language models may have their own preprocessing models, and that's why we should use the tokenizers trained along with that model. In our case, we are using distilBERT, so we should use the distilBERT tokenizer.  

With huggingface, loading different tokenizers is extremely easy: just import the AutoTokenizer from `transformers` and tell it what model you plan to use, and it will handle everything for you.

Reference:
 - [transformers.AutoTokenizer](https://huggingface.co/docs/transformers/master/en/model_doc/auto#transformers.AutoTokenizer)

In [9]:
import transformers

In [12]:
# [ TODO ] load the distilBERT tokenizer using AutoTokenizer

tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 9.38kB/s]
Downloading: 100%|██████████| 483/483 [00:00<00:00, 96.4kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 232kB/s]  
Downloading: 100%|██████████| 466k/466k [00:01<00:00, 383kB/s]  


#### Label processing

Our labels also need to be processed, so let's do that next.

For this tutorial, we'll use the OneHotEncoder provided by scikit-learn.

For now, just declare a new encoder and use `fit` to learn the data. Hint: you should still end up with 6 labels.

Documents:
 - [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder)

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [69]:
# [ TODO ] declare a new encoder and let it learn from the dataset

encoder = OneHotEncoder()

In [74]:
encoder.fit(dataset.data["test"].to_pandas())
encoder.fit(dataset.data["train"].to_pandas())

OneHotEncoder()

In [77]:
# check if you still have 6 labels
LABEL_COUNT = len(encoder.categories_[1])
print(LABEL_COUNT)

6


#### Process the data

To make things easier, we can write a function to process our dataset in batches. 

In [91]:
def preprocess(dataslice):
    
    """ Input: a batch of your dataset
        Example: { 'text': [['sentence1'], ['setence2'], ...],
                   'label': ['label1', 'label2', ...] }
    """
    
    # [ TODO ] use your tokenizor and encoder to get sentence embeddings and encoded labels
    encoded_str = tokenizer(string, padding=True, truncation=True) 
    
    """ Output: a batch of processed dataset
        Example: { 'input_ids': ...,
                   'attention_masks': ...,
                   'label': ... }
    """

In [ ]:
# map the function to the whole dataset
processed_data = dataset.map(preprocess,    # your processing function
                             batched = True # Process in batches so it can be faster
                            )

In [28]:
# print(processed_data)
# processed_data['train'][0]

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 20720
    })
})


{'text': 'My mother is having her car repaired.',
 'level': 'B1',
 'input_ids': [101, 2026, 2388, 2003, 2383, 2014, 2482, 13671, 1012, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'label': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]}

### DataCollator

You might have noticed that we skipped padding the sentences. That's because we are going to do it during training.  

To do training-time processing, we can use the DataCollator Class provided by `transformers`. And guess what - transformers has a class that will handle padding for us, too!

 - [transformers.DataCollatorWithPadding](https://huggingface.co/docs/transformers/master/en/main_classes/data_collator#transformers.DataCollatorWithPadding)

In [29]:
# [ TODO ] declare a collator to do padding during traning

data_collator = ...

## Training

Finally, we can move on to training.

### Preparation

We can load the pretrained model from `transformers`.  
Generally, you need to build your own model on top of BERT if you want to use BERT for some downstream tasks, but again, sequence classification is a popular topic. With the support from `transformers` library, it can be done in two lines of codes: 

1. Load `AutoModelForSequenceClassification` Class.
2. Load the pretrained model.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                           num_labels = LABEL_COUNT)

#### Split train/val data

The `Dataset` class we prepared before has a `train_test_split` method. You can use it to split your (processed) dataset.

Document:
 - [datasets.Dataset - Sort, shuffle, select, split, and shard](https://huggingface.co/docs/datasets/process.html#sort-shuffle-select-split-and-shard)

In [31]:
# [ TODO ] choose a validation size and split your data
train_val_dataset = ...

In [33]:
# print(train_val_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 18648
    })
    test: Dataset({
        features: ['text', 'level', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2072
    })
})


#### Setup training parameters

We are using the TrainerAPI to do the training. Trainer is yet another utility provided by huggingface, which helps you train the model with ease.  

Document:
- [transformers.TrainingArguments](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.TrainingArguments)
- [transformers.Trainer](https://huggingface.co/docs/transformers/master/en/main_classes/trainer#transformers.Trainer)

In [34]:
from transformers import TrainingArguments, Trainer

In [35]:
# [ TODO ] set and tune your training properties
OUTPUT_DIR = 
LEARNING_RATE = ...
BATCH_SIZE = ...
EPOCH = ...
training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCH,
    # you can set more parameters here if you want
)

# now give all the information to a trainer
trainer = Trainer(
    
    # set your parameters here

)

### Training

This is the easy part. Simply ask the trainer to train the model for you!

In [ ]:
trainer.train()

### Save for future use

Hint: try using `save_pretrained`

In [37]:
# [ TODO ] practice saving your model for future use


Configuration saved in model/finetuned/config.json
Model weights saved in model/finetuned/pytorch_model.bin


## Prediction

Now we know exactly how to train a model, but how do we use it for predicting results?

### Load finetuned model

In [ ]:
# [ TODO ] load the model that you saved

mymodel = ...

### Get the prediction

Here are a few example sentences:

In [39]:
examples = [
    # A2
    "Remember to write me a letter.",
    # B2
    "Strawberries and cream - a perfect combination.",
    "This so-called \"Perfect Evening\" was so disappointing, as well as discouraging us from coming to your Circle Theatre again.",
    # C1
    "Some may altogether give up their studies, which I think is a disastrous move.",
]

All we need to do is to transform them to embeddings, and then we can get predictions by calling your finetuned model.  

Since we don't have a DataCollator to pad the sentence and do the matrix transformation this time, we have to pad and transform the matrice on our own.

In [40]:
# Transform the sentences into embeddings
input = tokenizer(examples, truncation=True, padding=True, return_tensors="pt")
# Get the output
logits = mymodel(**input).logits
logits

tensor([[  3.4183,  -3.9602,  -4.7323,  -6.0863,  -5.3447,  -6.0282],
        [ -9.9472, -10.1832,  -9.0002,  -4.8197,   4.0705,  -4.7691],
        [-10.2303,  -8.6569,  -6.1997,   4.7033,  -5.0795,  -5.6949],
        [-11.8315, -11.4786,  -9.6928,  -0.7405,   0.0399,  -3.8067]],
       grad_fn=<AddmmBackward0>)

Logits aren't very readable for us. Let's use softmax 
activation to transform them into more probability-like numbers.

In [41]:
from torch import nn

predicts = nn.functional.softmax(logits, dim = -1)
predicts

tensor([[9.9878e-01, 6.2377e-04, 2.8820e-04, 7.4412e-05, 1.5621e-04, 7.8865e-05],
        [8.1670e-07, 6.4506e-07, 2.1055e-06, 1.3770e-04, 9.9971e-01, 1.4485e-04],
        [3.2685e-07, 1.5765e-06, 1.8400e-05, 9.9989e-01, 5.6403e-05, 3.0483e-05],
        [4.7224e-06, 6.7206e-06, 4.0086e-05, 3.0967e-01, 6.7584e-01, 1.4431e-02]],
       grad_fn=<SoftmaxBackward0>)

#### Transform logits back to labels

Now you've got the output. Write a function to map it back into labels!

In [42]:
# [ TODO ] try to process the result

(tensor([0, 4, 3, 4]), array(['A1', 'C1', 'B2', 'C1'], dtype='<U2'))

## Evaluation

Let's see how you did!  
Load the testing data and calculate your accuracy.

We want you to calculate the three kinds of accuracy mentioned in the lecture, which will also be explained in the following section.

In [ ]:
# [ TODO ] 
# load test data
# preprocess
# get predictions
# transform predictions back into labels

In [46]:
#  try printing out some predictions to check if the outputs are reasonable and if you need to adjust your model at the end of every step.

# for idx, (sent, level) in enumerate(zip(test_data['text'], predict_label)):
#     if idx >= 10: break
#     print(f'{level}: {sent}') 

C2: No longer a remote, backward, unimportant country, it became a force to be reckoned with in Europe.
B2: Unfortunately he was too fast and I couldn't keep up with him.
B2: Most mushrooms are totally harmless, but some are poisonous.
C2: This provided solid evidence that he committed the crime.
C1: You can't just accept everything you read in the newspapers at face value.
A1: Remember to write me a letter.
B1: She has long blond hair and blue eyes. She has a good figure.
B2: Nowadays the aim in clothing is not just for covering and protecting ourselves.
A2: Take two tablets, three times a day.
C2: Well, you will be if you saw our slide show and talk - members can hardly forget that relaxing afternoon when we unfolded the sails on the lake and enjoyed the tranquility of the area.


### Six Level Accuracy

Exact accuracy is probably what you're most familiar with:

$
accuracy = \frac{\#exactly\:the\:same\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
                    ^  ^     ^
```

The six level accuracy is $\frac{3}{6} = 0.5$

As the requirement, <u>your exact accuracy should be higher than $0.5$</u>.

In [ ]:
# [ TODO ] calculate accuracy

0.5617391304347826


### Three Level Accuracy

Three Level Accuracy is used when you only want a more general sense of right or wrong.

$
accuracy = \frac{\#the\:same\:ABC\:levels}{\#total}
$

Example:
```
Prediction:   A1 A2 B1 B2 C1 C2
Ground truth: A2 B1 B1 B2 B2 C2
              ^     ^  ^     ^
```

The three level accuracy is $\frac{4}{6} = 0.667$

As the requirement, <u>your exact accuracy should be higher than $0.6$</u>.

In [ ]:
# [ TODO ] calculate accuracy

0.6752173913043479


### Fuzzy accuracy

However, the level of a sentence is relatively subjective. Generally speaking, $\pm1$ errors are allowed in the real evaluation in linguistic area.  

For example, if the actual label is 'B1', we'll also consider the prediction 'right' if the model predicts 'B2' or 'A2'.

Hence, the fuzzy accuracy is

$
accuracy = \frac{\#good\:enough\:answers}{\#total}
$

Example:
```
Prediction:   0 1 2 3 4 5
Ground truth: 0 1 1 3 3 3
              ^ ^ ^ ^ ^
```

The fuzzy accuracy is $\frac{5}{6} = 0.833$

As the requirement, <u>your accuracy should be higher than $0.8$</u>.

In [ ]:
# [ TODO ] calculate accuracy

0.8665217391304347


## TA's Note

Congratulations, you made it to the end of the tutorial! Make sure you make an appointment to show your work and turn in your finished assignment before next week's lesson. We will ask you to run your code, so double check that everything is working and that your model is saved. Don't worry if you didn't pass the evaluation requirements, you'll still get partial points for trying.

## Appendix 


<a name="Appendix-1-Install-CUDA"></a>

### Appendix 1 - Install CUDA

1. Check your GPU vs. CUDA compatibility:
   - [NVIDIA -> Your GPU Compute Capability](https://developer.nvidia.com/cuda-gpus) -> GeForce and TITAN Products
2. Check library vs. CUDA compatibility: 
   - Pytorch: [Previous PyTorch Versions](https://pytorch.org/get-started/previous-versions/)
   - Tensorflow: [Linux/MacOX](https://www.tensorflow.org/install/source#tested_build_configurations) or [Windows](https://www.tensorflow.org/install/source_windows#tested_build_configurations)
3. Note the highest CUDA version that fits your system.

#### >> for conda/mamba users

You can directly install CUDA library with the selected CUDA version.
1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. `conda/mamba install -c conda-forge cudatoolkit=${VERSION}`

#### >> for non-conda users

1. Get [the driver for NVIDIA GPU](https://www.nvidia.com/download/index.aspx)
2. Download and install [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit-archive)
3. Download and install [cuDNN Library](https://developer.nvidia.com/rdp/cudnn-archive)

### Appendix 2 - Further Readings

1. [Huggingface Official Tutorials](https://github.com/huggingface/notebooks/tree/master/examples)
2. How to use Bert with other downstream tasks: [How to use BERT from the Hugging Face transformer library](https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209): 
3. Training with pytorch backend: [transformers-tutorials](https://github.com/abhimishra91/transformers-tutorials)
4. A more complicated example that include manual data/training processing with Pytorch: [Transformers for Multi-Label Classification made simple](https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1)
5. [Text Classification with tensorflow](https://github.com/huggingface/notebooks/blob/master/examples/text_classification-tf.ipynb): tensorflow example